## OpenAI LLM tracing using Maxim + Langchain

In [1]:

from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

True

In [2]:
import os

LOG_REPOSITORY_ID = os.getenv("MAXIM_LOG_REPO_ID")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

Here we will initialize MaximLangchainTracer

In [3]:
from maxim import Maxim, Config, LoggerConfig

logger = Maxim(Config()).logger(LoggerConfig(id=LOG_REPOSITORY_ID))

Setting up langchain call

In [5]:
from langchain_openai import ChatOpenAI
from maxim.logger.langchain import MaximLangchainTracer
import logging

# Set the logger level to debug for MaximSDK
logging.getLogger("MaximSDK").setLevel(logging.DEBUG)

langchain_tracer = MaximLangchainTracer(logger)

MODEL_NAME = "gpt-4o"
llm = ChatOpenAI(model=MODEL_NAME, api_key=OPENAI_API_KEY)

user_input = "Who was the first president of the India?"
system_message = "You are a helpful assistant."
# Make the API call to Claude using LangChain
messages = [("system", system_message), ("human", user_input)]
response = llm.invoke(messages,config={"callbacks":[langchain_tracer]})
response_text = response.content
print(response_text)

on_chat_model_start e0b45a4f-0925-48dd-bcf6-0396bf647fdd 8ab9c1f6-1177-42b6-bcb5-43d5cf084b8a None
llm_end e0b45a4f-0925-48dd-bcf6-0396bf647fdd 8ab9c1f6-1177-42b6-bcb5-43d5cf084b8a None
The first President of India was Dr. Rajendra Prasad. He served as President from January 26, 1950, when India became a republic, until May 13, 1962.
